# Alpha Diversity

## What is alpha diversity?

- Alpha diversity is a measure of the unique species (or taxonomic units) existing within one sample. 

Measurements of alpha diversity can be broken into two categories - richness and evenness. **Richness** is how most people think of diversity, as it describes the number of unique species in a sampling location. **Evenness**, on the other hand is a measure of how evenly the organisms in a community are distributed across the different species.


### An example

Suppose sampling location A harbors 10 unique species, and location B harbors 25 unique species. Location B would be considered to have higher richness than sample A.

However, if each of location A's species had a population of 2 (e.g. 2 foxes, 2 cats, 2 dogs, etc...), but location B was dominated by one species (e.g. 200 foxes, 1 cat, 1 dog, 1 horse, 1 proboscis monkey, 1 thorny dragon, etc...), we could say that location A has higher richness than location B.

If this isn't super clear yet, don't worry! There will be more examples as we work through this chapter.

## Why do we care about alpha diversity? (TODO)

When thinking about the health of a microbial community, it's important to consider factors that encourage stability, as a stable community is one that is not as susceptible to large perturbations by invading species or a lack of nutrients. 

[Konopka](https://www.nature.com/articles/ismej200988) outlines the importance of high alpha diversity for community stability, notably that high diversity encourages community resistance and resilience.

### Resistance


### Resilience
- **Functional diversity**
    - Functional diversity is the diversity (or richness) of functions that can be performed by a community.
    - High richness is associated with higher functional diversity.
- **Functional redundancy**
    - Functional redundancy is how well a community can keep performing a function if one individual or species that performs it is removed.
    - High richness and evenness are associated with functional redundancy.
    - 

## How do we quantify alpha diversity?



## Richness vs evenness


## Interactive example/Homework
